In [1]:
import re
import numpy as np
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import data_utils

## Import and merge data

In [8]:
# Import the data
orders = data_utils.clean_orders(data_path="../../data/")
orderlines = data_utils.clean_orderlines(data_path="../../data/")
products = data_utils.clean_products(data_path="../../data/")
brands = data_utils.clean_brands(data_path="../../data/")

# Merge the data
completed_sales =  data_utils.merge_data(orders, orderlines, products, brands)

5 missing values were removed from orders.
This represents 0.0022% of the data.


0 missing values were removed from orderlines.
This represents 0.00% of the data.


8792 missing values were removed from products
This represents 45.49% of the data.


0 missing values were removed from brands.
This represents 0.00% of the data.




## Explore corrupted decimal values
### Count decimal points in price values

In [4]:
prices = completed_sales[['orderline_id', 'total_paid', 'regular_price', 'promo_price', 'sale_price']].copy()

prices['regular_price_decimal_count'] = prices['regular_price'].str.count(r'\.')
prices['promo_price_decimal_count'] = prices['promo_price'].str.count(r'\.')
prices['sale_price_decimal_count'] = prices['sale_price'].str.count(r'\.')

prices

,orderline_id,total_paid,regular_price,promo_price,sale_price,regular_price_decimal_count,promo_price_decimal_count,sale_price_decimal_count
0,1398738,136.15,139.99,1.149.948,129.16,1,2,1
1,1529178,15.76,17.99,109.904,10.77,1,1,1
2,1181923,84.98,99.99,999.896,77.99,1,1,1
3,1276706,149.00,179,1.489.994,149.00,0,2,1
4,1154394,112.97,103.95,59.584,52.99,1,1,1
...,...,...,...,...,...,...,...,...
61667,1649446,18.98,35,13.99,13.99,0,1,1
61668,1649512,24.97,25,99.898,9.99,0,1,1
61669,1649522,24.97,25,99.898,9.99,0,1,1
61670,1649565,34.96,25,99.898,9.99,0,1,1


In [5]:
prices[['regular_price_decimal_count', 'promo_price_decimal_count', 'sale_price_decimal_count']].apply(pd.Series.value_counts)

,regular_price_decimal_count,promo_price_decimal_count,sale_price_decimal_count
0,18611,877,NaN
1,42264,40066,58120.0
2,797,20729,3552.0


<div class="alert alert-block alert-info">
    total_paid is the only price column which does not have corrupted data. 
    <br>
    It may be that we can use this column to determine the correct positions of the decimal points in the other price columns.
</div>